In [1]:
# import jupyter notebook version of dash framework
from jupyter_dash import JupyterDash as Dash
# import callbacks
from dash import Input, Output, State
# import html elements for dash
from dash import html
# Import markdown html translations
from dash import dcc
# Import warnings to ignore warnings
import warnings
warnings.filterwarnings('ignore')
# import os
import os

# Define image dataset path to use for app
image_dataset_path = "../data/testing"

# Define the subdirectories for the lesion types
subdirectories = ["no_tumor", "glioma", "meningioma", "pituitary"]

# Function to get image file names from a directory
def get_image_files(directory):
    files = os.listdir(directory)
    return [f for f in files if f.endswith('.jpg')]

# Create dropdown options from subdirectories
dropdown_options = [
    {"label": subdir.capitalize(), "value": subdir} 
    for subdir in subdirectories
]

# Define style sheet 
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Create the dash app
app = Dash(__name__, external_stylesheets=external_stylesheets)

# Create markdown for our dashboard
markdown_1 = dcc.Markdown("""
    # Brain Tumor Lesion Assesment
""")

markdown_2 = dcc.Markdown("""
    The model integrated in into this dashbaord has the capability to predict different brian lesions.
    The four supported brain lession classification are Meningioma, Pituitary, Glioma tumors. 
    The fourth possible prediction would be a No Tumor classification.
""")

# Upload image
upload_img = dcc.Upload(
    id='upload-image',
    children=html.Div([
        'Drag and Drop or ',
        html.A('Select File')
    ]),
        style={
        'width': '100%',
        'height': '60px',
        'lineHeight': '60px',
        'borderWidth': '1px',
        'borderStyle': 'dashed',
        'borderRadius': '5px',
        'textAlign': 'center',
        'margin': '10px'
    },
    multiple=True, # Do not allow multiple uploads
)

# Set the layout
app.layout = html.Div(children=[
    html.Div(markdown_1),
    html.Div(markdown_2),
    html.Div([upload_img, html.Div(id='output-image-upload')]),
])

# function to parse file path 
def parse_contents(contents, filename):
    return html.Div([
        html.H5(filename),

        # HTML images accept base64 encoded strings in the same format
        # that is supplied by the upload
        html.Img(src=contents),
        html.Hr()
    ])
# Define callback 
@app.callback(Output('output-image-upload', 'children'),
                Input('upload-image', 'contents'),
                State('upload-image', 'filename'))
# Define an update function
def update_output(list_of_contents, list_of_names):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n) for c, n in
            zip(list_of_contents, list_of_names)]
        return children

if __name__ == '__main__':
    app.run_server(mode='external', host='localhost', port=5000)

Dash app running on http://localhost:5000/


In [3]:
# from dash import Dash, dcc, html, Input, Output, State, callback

# import datetime

# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# app = Dash(__name__, external_stylesheets=external_stylesheets)

# app.layout = html.Div([
#     dcc.Upload(
#         id='upload-image',
#         children=html.Div([
#             'Drag and Drop or ',
#             html.A('Select Files')
#         ]),
#         style={
#             'width': '100%',
#             'height': '60px',
#             'lineHeight': '60px',
#             'borderWidth': '1px',
#             'borderStyle': 'dashed',
#             'borderRadius': '5px',
#             'textAlign': 'center',
#             'margin': '10px'
#         },
#         # Allow multiple files to be uploaded
#         multiple=True
#     ),
#     html.Div(id='output-image-upload'),
# ])

# def parse_contents(contents, filename, date):
#     return html.Div([
#         html.H5(filename),
#         html.H6(datetime.datetime.fromtimestamp(date)),

#         # HTML images accept base64 encoded strings in the same format
#         # that is supplied by the upload
#         html.Img(src=contents),
#         html.Hr(),
#         html.Div('Raw Content'),
#         html.Pre(contents[0:200] + '...', style={
#             'whiteSpace': 'pre-wrap',
#             'wordBreak': 'break-all'
#         })
#     ])

# @callback(Output('output-image-upload', 'children'),
#               Input('upload-image', 'contents'),
#               State('upload-image', 'filename'),
#               State('upload-image', 'last_modified'))
# def update_output(list_of_contents, list_of_names, list_of_dates):
#     if list_of_contents is not None:
#         children = [
#             parse_contents(c, n, d) for c, n, d in
#             zip(list_of_contents, list_of_names, list_of_dates)]
#         return children

# if __name__ == '__main__':
#     app.run(debug=True)

In [4]:
app.run_server(mode="external", height=500, host="localhost", port=5000)